<a href="https://colab.research.google.com/github/prathamnishad/experimental/blob/master/intern2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import zipfile
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing



local_zip = '/content/drive/My Drive/raw.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
raw=os.path.join("/tmp/data/raw")
train_gyro_phone=os.path.join("/tmp/data/raw/train/phone/gyro")
print('total train file:', len(os.listdir(train_gyro_phone)))
train_accel_phone=os.path.join("/tmp/data/raw/train/phone/accel")
print('total train file:', len(os.listdir(train_accel_phone)))
train_gyro_watch=os.path.join("/tmp/data/raw/train/watch/gyro")
print('total train file:', len(os.listdir(train_gyro_watch)))
train_accel_watch=os.path.join("/tmp/data/raw/train/watch/accel")
print('total train file:', len(os.listdir(train_accel_watch)))

test_gyro_phone=os.path.join("/tmp/data/raw/test/phone/gyro")
print('total test file:', len(os.listdir(test_gyro_phone)))
test_accel_phone=os.path.join("/tmp/data/raw/test/phone/accel")
print('total test file:', len(os.listdir(test_accel_phone)))
test_gyro_watch=os.path.join("/tmp/data/raw/test/watch/gyro")
print('total test file:', len(os.listdir(test_gyro_watch)))
test_accel_watch=os.path.join("/tmp/data/raw/test/watch/accel")
print('total test file:', len(os.listdir(test_accel_watch)))


total train file: 20
total train file: 20
total train file: 20
total train file: 20
total test file: 14
total test file: 14
total test file: 14
total test file: 14


In [3]:
#for train
df=pd.DataFrame()
for n in os.listdir(train_gyro_phone):
    a=pd.read_csv(os.path.join(train_gyro_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
for n in os.listdir(train_accel_phone):
    a=pd.read_csv(os.path.join(train_accel_phone,n),sep=",",header=None)
    df=pd.concat([df,a])
  
for n in os.listdir(train_gyro_watch):
    a=pd.read_csv(os.path.join(train_gyro_watch,n),sep=",",header=None)
    df=pd.concat([df,a])

for n in os.listdir(train_accel_watch):
    a=pd.read_csv(os.path.join(train_accel_watch,n),sep=",",header=None)
    df=pd.concat([df,a])
d_train=df
df.head()

,0,1,2,3,4,5
0,1612,A,3952189047449,0.817322,-5.137268,-0.20063782;
1,1612,A,3952239401453,-1.029923,-3.979431,-1.3396301;
2,1612,A,3952289755457,0.493454,-1.189850,-2.666687;
3,1612,A,3952340109460,2.300003,-2.157044,-1.5438995;
4,1612,A,3952390463464,1.797806,-2.138672,1.0609283;


In [4]:
#for test
df_test=pd.DataFrame()
for n in os.listdir(test_gyro_phone):
    a=pd.read_csv(os.path.join(test_gyro_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
for n in os.listdir(test_accel_phone):
    a=pd.read_csv(os.path.join(test_accel_phone,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
  
for n in os.listdir(test_gyro_watch):
    a=pd.read_csv(os.path.join(test_gyro_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])

for n in os.listdir(test_accel_watch):
    a=pd.read_csv(os.path.join(test_accel_watch,n),sep=",",header=None)
    df_test=pd.concat([df_test,a])
d_test=df_test
df_test.head()

,0,1,2,3,4,5
0,1629,A,1127073874136682,-0.664368,-0.642181,-0.3517151;
1,1629,A,1127073924490686,-0.154068,-0.731796,-0.33454895;
2,1629,A,1127073974844690,0.290558,-0.411285,-0.12980652;
3,1629,A,1127074025198693,0.679916,-0.354401,-0.27474976;
4,1629,A,1127074075552697,0.757401,-0.079544,-0.45448303;


In [0]:
#function to preprocess data
def make_df(df):
  df.columns=["id","activity","time","x","y","z"]
  df['z'] = df['z'].apply(lambda x: float(x[:-1]))
  df['time'] = df['time'].apply(lambda x: x//1000000)
  df['time'] = df['time'].apply(lambda x: datetime.fromtimestamp(x))
  for n in ("month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"):
      df[n]=getattr(df.time.dt,n)
  df.drop(columns="time",inplace=True)

  for n in ["is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]:
    df[n]=df[n].apply(lambda x:int(x))

  act=["A","B","C","D","E","F","G","H","I","J","K","L","M","O","P","Q","R","S"]
  df["activity"]=df["activity"].apply(lambda a: act.index(a))
  df = df.sample(frac=1).reset_index(drop=True)
  df.drop(columns="id",inplace=True)
  return df
  

In [0]:
#for train
df_train=make_df(df)
#for test
df_test=make_df(df_test)

In [0]:
dfa=pd.DataFrame()
dfa=pd.concat([dfa,df_test])
dfa=pd.concat([dfa,df_train])

In [8]:

mm_scaler = preprocessing.MinMaxScaler()
dfa[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]] = mm_scaler.fit_transform(dfa[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]])
dfa.describe()

,activity,x,y,z,month,week,day,dayofweek,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
count,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06,9.841341e+06
mean,8.519208e+00,5.356734e-01,3.878899e-01,5.478616e-01,4.213743e-01,4.212925e-01,4.819992e-01,5.008542e-01,4.239614e-01,3.205742e-02,3.319609e-02,1.155869e-02,1.141420e-02,4.071092e-03,3.455931e-03
std,5.189590e+00,6.034050e-02,5.829600e-02,5.923248e-02,3.001880e-01,2.778723e-01,2.930207e-01,3.329557e-01,2.772879e-01,1.761526e-01,1.791483e-01,1.068882e-01,1.062258e-01,6.367510e-02,5.868550e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,5.170561e-01,3.606420e-01,5.298223e-01,1.818182e-01,1.923077e-01,2.333333e-01,1.666667e-01,1.945205e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.000000e+00,5.328251e-01,4.001552e-01,5.364136e-01,3.636364e-01,3.846154e-01,4.666667e-01,5.000000e-01,3.863014e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.300000e+01,5.492285e-01,4.036007e-01,5.657928e-01,6.363636e-01,6.346154e-01,7.333333e-01,8.333333e-01,6.356164e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.700000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [0]:
train_y=dfa["activity"]
train_x=dfa.drop(columns="activity")
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y , train_size = 0.8, random_state =  90)
train=len(x_train)
test=len(x_test)
mm_scaler = preprocessing.MinMaxScaler().fit(x_train)

In [10]:
x_train[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]] = mm_scaler.transform(x_train[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]])

x_test[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]] = mm_scaler.transform(x_test[["x","y","z","month","week","day","dayofweek","dayofyear","is_month_end","is_month_start","is_quarter_end","is_quarter_start","is_year_end","is_year_start"]])
 
x_test.describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,x,y,z,month,week,day,dayofweek,dayofyear,is_month_end,is_month_start,is_quarter_end,is_quarter_start,is_year_end,is_year_start
count,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06,1.968269e+06
mean,5.356960e-01,3.878459e-01,5.478420e-01,4.213781e-01,4.212534e-01,4.817432e-01,5.013300e-01,4.239452e-01,3.187979e-02,3.316620e-02,1.153094e-02,1.147048e-02,4.132565e-03,3.432458e-03
std,6.041575e-02,5.828814e-02,5.926973e-02,3.001061e-01,2.777612e-01,2.929001e-01,3.329103e-01,2.771869e-01,1.756801e-01,1.790704e-01,1.067614e-01,1.064844e-01,6.415208e-02,5.848656e-02
min,6.632729e-02,8.948367e-04,3.507460e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.170811e-01,3.606447e-01,5.298632e-01,1.818182e-01,1.923077e-01,2.333333e-01,1.666667e-01,1.945205e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.328245e-01,4.001602e-01,5.364126e-01,3.636364e-01,3.846154e-01,4.666667e-01,5.000000e-01,3.863014e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.492127e-01,4.035713e-01,5.657721e-01,6.363636e-01,6.346154e-01,7.333333e-01,8.333333e-01,6.356164e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.593488e-01,8.693217e-01,8.511969e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [24]:


model_tree = tree.DecisionTreeClassifier()

start = time.time()
model_tree.fit(x_train, y_train)
end = time.time()
print(end - start)

y_pred= model_tree.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy of %s is %s"%(model_tree, acc))


161.18219900131226
Accuracy of DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') is 0.9027465249922648


In [11]:

classifier = RandomForestClassifier(n_estimators = 50)
start = time.time()
classifier.fit(x_train, y_train)
end = time.time()
print(end - start)
y_pred = classifier.predict(x_test)
result = accuracy_score(y_test,y_pred)
print("Accuracy:",result)

1692.355530500412
Accuracy: 0.9357933290622369
